In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
import tensorflow as tf
import torch
import torch.nn as nn

class Net(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, output_dim)
        )

    def forward(self, x):
        return self.net(x)

# === Przygotowanie danych ===
# Wczytaj dane, aby uzyskać te same skalowanie i kodowanie etykiet
data = pd.read_excel("złączone_dane.xlsx")
data = data.drop("image_id", axis=1)
data = data.drop(columns=[col for col in data.columns if any(x in col for x in ['3_p', '4_p', '5_p'])])

X = data.drop("label", axis=1)
y = data["label"]

# Dopasuj LabelEncoder i StandardScaler
le = LabelEncoder()
y_encoded = le.fit_transform(y)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# === Przykładowy nowy rekord (np. pierwszy z oryginalnego zbioru) ===
sample = X.iloc[0].values.reshape(1, -1)
sample_scaled = scaler.transform(sample)

# === TensorFlow model ===
model_tf = tf.keras.models.load_model("models/NeuralNet_TF.h5")
pred_tf = np.argmax(model_tf.predict(sample_scaled), axis=1)
pred_label_tf = le.inverse_transform(pred_tf)
print(f"🧠 TensorFlow - przewidziana klasa: {pred_label_tf[0]}")

# === PyTorch model ===
input_dim = X.shape[1]
output_dim = len(le.classes_)
model_pt = Net(input_dim, output_dim)
model_pt.load_state_dict(torch.load("models/NeuralNet_PT.pt"))
model_pt.eval()

with torch.no_grad():
    sample_tensor = torch.tensor(sample_scaled, dtype=torch.float32)
    pred_pt = model_pt(sample_tensor)
    pred_class_pt = torch.argmax(pred_pt, axis=1).item()
    pred_label_pt = le.inverse_transform([pred_class_pt])
    print(f"🧠 PyTorch - przewidziana klasa: {pred_label_pt[0]}")



c:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
🧠 TensorFlow - przewidziana klasa: a+
🧠 PyTorch - przewidziana klasa: a+
